In [1]:
import numpy as np
import pandas as pd

In [2]:
pitching_df = pd.read_csv("db/pitching4.csv")

In [3]:
pitching_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 8 columns):
playerID      221 non-null object
G             221 non-null int64
W             221 non-null int64
SO            221 non-null int64
PCT           221 non-null float64
ERA           221 non-null float64
WHIP          221 non-null float64
HOF_member    221 non-null int64
dtypes: float64(3), int64(4), object(1)
memory usage: 13.9+ KB


In [4]:
X = pitching_df[["G","W", "SO","PCT","ERA"]]
y = pitching_df["HOF_member"]

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [30]:
# split data into test and train
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)


In [31]:
# Normalize the data.  Here, we are using StandardScaler
X_scaler = StandardScaler().fit(X_train)
#X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)



C:\Users\Laptop 1\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Laptop 1\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """
C:\Users\Laptop 1\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


In [32]:
model = LogisticRegression(C=1.0, penalty='l2', random_state=27, solver="saga", max_iter=5000)

In [33]:
X_train_scaled.shape

(165, 5)

In [34]:
X_test_scaled.shape

(56, 5)

In [35]:
model.fit(X_train_scaled,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=5000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=27, solver='saga',
          tol=0.0001, verbose=0, warm_start=False)

In [36]:
from sklearn.metrics import mean_squared_error

In [37]:
predictions = model.predict(X_test_scaled)
MSE = mean_squared_error(y_test, predictions)
r2 = model.score(X_test_scaled, y_test)

print(f"MSE: {MSE}, R2: {r2}") 


MSE: 0.07142857142857142, R2: 0.9285714285714286


If curious about which players in the test split were predicted for what, we can take these two outputs, import and pair them

In [47]:
X_test["predictions"] = predictions
X_test.head()

C:\Users\Laptop 1\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,G,W,SO,PCT,ERA,predictions
132,259,84,645,0.518519,3.387956,0
148,228,107,792,0.625731,3.041096,0
93,240,136,2306,0.719577,2.585436,1
180,250,141,1675,0.681159,2.287124,1
15,222,99,600,0.600000,3.222247,0


In [48]:
X_test.to_csv("db/Xtest.csv", index=False, header=True)

Here's what we've been waiting for:  Let's make Hall-Of-Fame predictions for more recent and current players.

In [38]:
recent_df = pd.read_csv("db/pitching_recent.csv")

In [39]:
recent_players = recent_df["playerID"]
X_recent = recent_df[["G","W","SO","PCT","ERA"]]


In [40]:
X_recent_scaled = X_scaler.transform(X_recent)

C:\Users\Laptop 1\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


In [41]:
predictions_recent = model.predict(X_recent_scaled)

In [42]:
X_recent["predictions"] = predictions_recent
X_recent["playerID"] = recent_players

C:\Users\Laptop 1\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Laptop 1\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [43]:
people_df = pd.read_csv("db/people.csv")

In [44]:
players_df = people_df[["playerID","nameFirst","nameLast"]]
players_df["nameFull"] = players_df["nameFirst"]+' '+players_df["nameLast"]
players_df.head()

C:\Users\Laptop 1\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,playerID,nameFirst,nameLast,nameFull
0,aardsda01,David,Aardsma,David Aardsma
1,aaronha01,Hank,Aaron,Hank Aaron
2,aaronto01,Tommie,Aaron,Tommie Aaron
3,aasedo01,Don,Aase,Don Aase
4,abadan01,Andy,Abad,Andy Abad


In [45]:
X_recent.to_csv("db/Xrecent.csv", index=False, header=True)

In [46]:
Predicted_for_HOF = X_recent.loc[X_recent["predictions"] == 1, :]
Predicted_for_HOF = pd.merge(Predicted_for_HOF, players_df, on="playerID", how="left")
del Predicted_for_HOF["nameFirst"]
del Predicted_for_HOF["nameLast"]
display(Predicted_for_HOF)

,G,W,SO,PCT,ERA,predictions,playerID,nameFull
0,233,134,1357,0.705263,2.820024,1,hallaro01,Roy Halladay
1,207,118,1623,0.742138,2.095041,1,kershcl01,Clayton Kershaw
